In [4]:
import os, tqdm

In [5]:
from sklearn.metrics import r2_score
import numpy as np

# metric
def metric(label, pred):
    assert label.shape == pred.shape
    
    with np.errstate(divide = 'ignore', invalid = 'ignore'):
        mask = (label == label) & (pred == pred)
        mask = mask.astype(np.float32)
        mask /= np.mean(mask)
        
        male = np.abs(np.subtract(np.log(pred), np.log(label))).astype(np.float32)
        mae = np.abs(np.subtract(pred, label)).astype(np.float32)
        
        male = np.nan_to_num(male * mask)
        male = np.mean(male)
        
        mae = np.nan_to_num(mae * mask)
        mae = np.mean(mae)
        
        rmse = np.square(mae)
        rmse = np.nan_to_num(rmse * mask)
        rmse = np.sqrt(np.mean(rmse))
        
        mape = np.divide(mae, label)
        mape = np.nan_to_num(mape * mask)
        mape = np.median(mape*mask)
        
        print('masked:', np.sum(mask == 0))
    return male, rmse, mape

In [6]:
os.listdir('.')

['house-dataset-osm-road2vec-poa.ipynb',
 'house-dataset-visualization.html',
 'house-dataset-visualization.ipynb',
 '.ipynb_checkpoints',
 'house-lgbm.ipynb',
 'generateSE.py',
 'sp',
 'house-dataset-visualization-Copy1.ipynb',
 'house-dataset-osm-road2vec-dbscan.ipynb',
 'node2vec.py',
 '.gitkeep ',
 'poa',
 'house-dataset-osm-road2vec-Copy2.ipynb',
 'fc',
 'kc',
 'house-dataset-osm-road2vec-kc.ipynb',
 'house_reverse_geocoding.py',
 '__pycache__',
 'house-lgbm-sonia.ipynb',
 'house-dataset-osm-road2vec-Copy1.ipynb',
 'osmdata',
 'house-dataset-osm-road2vec.ipynb',
 'house-dataset-osm-buildings.ipynb',
 'house-dataset-osm-buildings-Copy1.ipynb',
 'cache']

In [7]:
datasets = ['poa']

In [8]:
import numpy as np

In [9]:
streetmap = {
    'style': 'mapbox://styles/mapbox/streets-v9',
    'token': 'pk.eyJ1IjoiaHNtNjkxMSIsImEiOiJjazl0and6aDUwOWF2M2RvemdrYjllczV3In0.qGmaAF6v-1LAF9C-dnMLBg'
}
mybasemap = {
    #'style': 'mapbox://styles/mapbox/streets-v9',
    'style': 'mapbox://styles/mapbox/satellite-v9',
    'token': 'pk.eyJ1IjoiaHNtNjkxMSIsImEiOiJjazl0and6aDUwOWF2M2RvemdrYjllczV3In0.qGmaAF6v-1LAF9C-dnMLBg'
}

In [10]:
from cartoframes.viz import *

In [11]:
import pandas as pd

In [12]:
import geopandas as gpd

In [13]:
for dname in ['poa']:#['kc', 'fc', 'sp', 'poa']:
    print(dname)
    data = np.load(f'{dname}/data.npz')
    
    dict1 = {'lat':data['X_train'][:,0], 'lng': data['X_train'][:,1], 'price': data['y_train']}
    dict2 = {'lat':data['X_test'][:,0], 'lng': data['X_test'][:,1], 'price': data['y_test']}
    attr_names = []
    for a in range(2, data['X_train'].shape[1]):
        dict1.update({f'attr{a-2}': data['X_train'][:, a]})
        dict2.update({f'attr{a-2}': data['X_test'][:, a]})
        attr_names.append(f'attr{a-2}')
    df1 = pd.DataFrame(dict1)
    df2 = pd.DataFrame(dict2)
    df = pd.concat([df1, df2])
    
    train_gdf = gpd.GeoDataFrame(df1.copy(), geometry=gpd.points_from_xy(x=df1.lng, y=df1.lat))
    train_gdf.crs = 'EPSG:4326'
    test_gdf = gpd.GeoDataFrame(df2.copy(), geometry=gpd.points_from_xy(x=df2.lng, y=df2.lat))
    test_gdf.crs = 'EPSG:4326'
    house_gdf = gpd.GeoDataFrame(df.copy(), geometry=gpd.points_from_xy(x=df.lng, y=df.lat))
    house_gdf.crs = 'EPSG:4326'
    #print(np.exp(df['price'].values).mean())
    gdf = house_gdf
    for attr in attr_names:
        print(attr, gdf[attr].nunique())
        if gdf[attr].nunique() < 30:
            gdf[attr] = gdf[attr].astype(str)
    gdfcpy = gdf.copy()
    
    
#     display(Map(
#         [
#             Layer(gdfcpy, color_category_style(tattr, cat=cat, palette='cb_blues'), encode_data=False),
#             Layer(gdf, color_continuous_style('price', palette='sunset'), encode_data=False),
#         ],
#         basemap=mybasemap))
    
    break

poa
attr0 9
attr1 2644
attr2 10
attr3 2
attr4 2


In [14]:
import osmnx as ox
from shapely.geometry import *

if not os.path.isdir('osmdata'):
    os.mkdir('osmdata')
    
DATASET_NAME = dname
OSM_FILE_PATH = f'osmdata/{DATASET_NAME}.graphml'

from shapely.geometry import MultiPoint


x1, y1, x2, y2 = gdf.total_bounds

house_center_latitude = (y1 + y2)/2 #sensor_hull.centroid.y
house_center_longitude = (x1 + x2)/2 #sensor_hull.centroid.x

     
graphs = dict()
# retrieve the street network for the location
if not os.path.isfile(OSM_FILE_PATH):
    center_point = gpd.GeoDataFrame(geometry = [Point(house_center_longitude, house_center_latitude)])
    center_point.crs = 'epsg:4326'
    center_point = center_point.to_crs('epsg:3310')
    max_distance = gdf.to_crs('epsg:3310').distance(center_point.iloc[0].geometry).max()+1000
    print('max_distance:', max_distance)
    graph = ox.graph_from_point((house_center_latitude, house_center_longitude), dist=max_distance)

    # save the street network to a shapefile
    ox.save_graphml(graph, filepath=OSM_FILE_PATH)
else:
    graph = ox.load_graphml(filepath=OSM_FILE_PATH)
    

# buildings = buildings.reset_index()
# buildings.geometry = buildings.geometry.centroid

In [15]:
# graph2 = ox.graph_from_place('vashon')
# osm_nodes2, osm_edges2 = ox.graph_to_gdfs(graph2)
# osm_nodes1, osm_edges1 = ox.graph_to_gdfs(graph)

# osm_nodes = pd.concat((osm_nodes1, osm_nodes2))
# osm_edges = pd.concat((osm_edges1, osm_edges2))

osm_nodes, osm_edges = ox.graph_to_gdfs(graph)

In [16]:



osm_nodes['osmidn'] = osm_nodes.index
osm_nodes['osmidstr'] = osm_nodes['osmidn'].astype(str)
osm_edges = osm_edges.reset_index()
cond = np.array([str(type(s)) for s in osm_edges['highway']]) == "<class 'str'>"
osm_edges = osm_edges[cond]

In [17]:
alist = osm_nodes.geometry.tolist()

In [18]:
np.random.shuffle(alist)

In [19]:
Layer(gpd.GeoDataFrame(geometry=alist[:1000]))

In [20]:
center_point = gpd.GeoDataFrame(geometry = [Point(house_center_longitude, house_center_latitude)])
center_point.crs = 'epsg:4326'
center_point = center_point.to_crs('epsg:3310')
max_distance = gdf.to_crs('epsg:3310').distance(center_point.iloc[0].geometry).max()+1000
buildings = ox.geometries.geometries_from_point((house_center_latitude, house_center_longitude), 
                                    tags = {'building': True},
                                    dist=max_distance)

In [21]:
rbuildings = buildings.reset_index()
fbuildings = buildings.reset_index()
fbuildings.geometry = fbuildings.geometry.centroid
fbuildings['nx'] = fbuildings.geometry.x
fbuildings['ny'] = fbuildings.geometry.y

fbuildings['barea'] = buildings.reset_index().to_crs('epsg:3310').area

corr_osmid = []
for _, htem in tqdm.tqdm(house_gdf.iterrows(), total=len(house_gdf)):
    ffbs = rbuildings[(fbuildings['nx'] > htem.lng - 0.002) & (fbuildings['nx'] < htem.lng + 0.002) & \
                        (fbuildings['ny'] > htem.lat - 0.002) & (fbuildings['ny'] < htem.lat + 0.002) & \
                         (fbuildings['barea'] > 70)]
    if len(ffbs) == 0:
        corr_osmid.append(-1)
    else:
        target_bd = ffbs.iloc[ffbs.distance(htem.geometry).values.argmin()]
        corr_osmid.append(target_bd['osmid'])

nhouse_gdf = house_gdf.copy()
nhouse_gdf['osmid'] = corr_osmid

/tmp/ipykernel_1320774/565877014.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  fbuildings.geometry = fbuildings.geometry.centroid
  0%|                                                                                         | 0/15368 [00:00<?, ?it/s]/tmp/ipykernel_1320774/565877014.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  target_bd = ffbs.iloc[ffbs.distance(htem.geometry).values.argmin()]
100%|████████████████████████████████████████████████████████████████████████████| 15368/15368 [02:14<00:00, 114.61it/s]


In [22]:
osmid2geo = {osmid:geo for osmid, geo in zip(rbuildings['osmid'], rbuildings['geometry'])}

geos = []
for i, osmid in enumerate(nhouse_gdf['osmid']):
    if osmid > 0:
        geos.append(osmid2geo[osmid])
    else:
        geos.append(house_gdf.iloc[i].geometry)

nhouse_gdf.geometry = geos
nhouse_gdf.geometry = nhouse_gdf.geometry.centroid

/tmp/ipykernel_1320774/2617699450.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nhouse_gdf.geometry = nhouse_gdf.geometry.centroid


# Next

In [23]:
import node2vec
import numpy as np
import networkx as nx
from gensim.models import Word2Vec

with open(f'{dname}/Adj.txt', 'w') as fp:
    for _, item in osm_edges.iterrows():
        fp.write(f'{item.u} {item.v} 1.0\n')
        
        
def read_graph(edgelist):
    G = nx.read_edgelist(
        edgelist, nodetype=str, data=(('weight',float),),
        create_using=nx.DiGraph())
    return G

Adj_file = f'{dname}/Adj.txt'
nx_G = read_graph(Adj_file)

num_walks = 10
walk_length = 20
p = 2
q = 1
is_directed = True

G = node2vec.Graph(nx_G, is_directed, p, q)
G.preprocess_transition_probs()
node2vec_walks = G.simulate_walks(num_walks, walk_length)

/disk1/jupyter/smhan/ASI/datasets/node2vec.py:120: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  J = np.zeros(K, dtype=np.int)


Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


In [24]:
from gensim.models import Word2Vec

vector_size = 32
sentences = node2vec_walks
model_node = Word2Vec(sentences, window=5, min_count=0, workers=4, vector_size=vector_size)

edge_discover_path_list = []
for path in node2vec_walks:# + discover_path_list:
    edge = []
    for u, v in zip(path[:-1], path[1:]):
        edge.append(f'{u}-{v}')
    edge_discover_path_list.append(edge)
sentences = edge_discover_path_list
model_edge = Word2Vec(sentences, window=5, min_count=1, workers=4, vector_size=vector_size)

collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #10000, processed 199671 words, keeping 55101 word types
PROGRESS: at sentence #20000, processed 399352 words, keeping 65226 word types
PROGRESS: at sentence #30000, processed 599087 words, keeping 67743 word types
PROGRESS: at sentence #40000, processed 798747 words, keeping 68467 word types
PROGRESS: at sentence #50000, processed 998531 words, keeping 68663 word types
PROGRESS: at sentence #60000, processed 1198205 words, keeping 68740 word types
PROGRESS: at sentence #70000, processed 1397806 words, keeping 68767 word types
PROGRESS: at sentence #80000, processed 1597515 words, keeping 68767 word types
PROGRESS: at sentence #90000, processed 1797154 words, keeping 68767 word types
PROGRESS: at sentence #100000, processed 1996876 words, keeping 68767 word types
PROGRESS: at sentence #110000, processed 2196647 words, keeping 68767 word types
PROGRESS: at senten

EPOCH 1 - PROGRESS: at 16.88% examples, 2310951 words/s, in_qsize 7, out_qsize 0
EPOCH 1 - PROGRESS: at 34.27% examples, 2346233 words/s, in_qsize 7, out_qsize 0
EPOCH 1 - PROGRESS: at 51.96% examples, 2373086 words/s, in_qsize 7, out_qsize 0
EPOCH 1 - PROGRESS: at 70.15% examples, 2403882 words/s, in_qsize 7, out_qsize 0
EPOCH 1 - PROGRESS: at 88.99% examples, 2439374 words/s, in_qsize 7, out_qsize 0
EPOCH 1: training on 13732160 raw words (13732160 effective words) took 5.6s, 2462465 effective words/s
EPOCH 2 - PROGRESS: at 19.14% examples, 2625864 words/s, in_qsize 7, out_qsize 0
EPOCH 2 - PROGRESS: at 39.44% examples, 2702615 words/s, in_qsize 7, out_qsize 0
EPOCH 2 - PROGRESS: at 60.18% examples, 2747772 words/s, in_qsize 7, out_qsize 0
EPOCH 2 - PROGRESS: at 81.28% examples, 2783191 words/s, in_qsize 7, out_qsize 0
EPOCH 2: training on 13732160 raw words (13732160 effective words) took 4.9s, 2809889 effective words/s
EPOCH 3 - PROGRESS: at 20.52% examples, 2817674 words/s, in_qsi

Creating a fresh vocabulary
Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 164646 unique words (100.00% of original 164646, drops 0)', 'datetime': '2023-08-21T18:46:57.354523', 'gensim': '4.3.1', 'python': '3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:26:04) [GCC 10.4.0]', 'platform': 'Linux-5.15.0-76-generic-x86_64-with-glibc2.31', 'event': 'prepare_vocab'}
Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 13044490 word corpus (100.00% of original 13044490, drops 0)', 'datetime': '2023-08-21T18:46:57.355037', 'gensim': '4.3.1', 'python': '3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:26:04) [GCC 10.4.0]', 'platform': 'Linux-5.15.0-76-generic-x86_64-with-glibc2.31', 'event': 'prepare_vocab'}
deleting the raw counts dictionary of 164646 items
sample=0.001 downsamples 0 most-common words
Word2Vec lifecycle event {'msg': 'downsampling leaves estimated 13044490 word corpus (100.0%% of prior 13044490)', 'datetime': '2023-08-21T18:46:57.

In [25]:
magnitudes = []
for osmid in osm_nodes['osmidstr']:
    if osmid in model_node.wv:
        vec = model_node.wv[osmid]
        mag = np.linalg.norm(vec)
        magnitudes.append(mag)
    else:
        magnitudes.append(-1)
osm_nodes['magnitude'] = magnitudes
# Layer(osm_nodes_3310[osm_nodes_3310['magnitude'] > 0], color_continuous_style('magnitude'))


magnitudes = []
osm_edges['u-v'] = osm_edges['u'].astype(str) + '-' + osm_edges['v'].astype(str)
for osmid in osm_edges['u-v']:
    if osmid in model_edge.wv:
        vec = model_edge.wv[osmid]
        mag = np.linalg.norm(vec)
        magnitudes.append(mag)
    else:
        magnitudes.append(-1)
osm_edges['magnitude'] = magnitudes
# Layer(osm_edges_3310[osm_edges_3310['magnitude'] > 0], color_continuous_style('magnitude'))

In [26]:
# ftest_edges = osm_edges_3310[osm_edges_3310['magnitude'] > 0].copy()
# ftest_edges['bx'] = ftest_edges.geometry.centroid.x
# ftest_edges['by'] = ftest_edges.geometry.centroid.y

# house_gdf_3310 = nhouse_gdf.to_crs('epsg:3310')
# house_gdf_3310['bx'] = house_gdf_3310.geometry.centroid.x
# house_gdf_3310['by'] = house_gdf_3310.geometry.centroid.y

# vectors = []
# for _, item in tqdm.tqdm(house_gdf_3310.iterrows(), total=len(house_gdf)):
#     iftest_edges = ftest_edges[(ftest_edges['bx'] > item['bx'] - 500) & 
#                 (ftest_edges['bx'] < item['bx'] + 500) & 
#                 (ftest_edges['by'] > item['by'] - 500) & 
#                 (ftest_edges['by'] < item['by'] + 500)]
    
#     jtem = iftest_edges.iloc[iftest_edges.distance(item.geometry).argmin()]
#     vec = model.wv[jtem['u-v']]
#     vectors.append(vec)

In [27]:
ftest_nodes = osm_nodes[osm_nodes['magnitude'] > 0].to_crs('epsg:3310')
ftest_nodes['bx'] = ftest_nodes.geometry.centroid.x
ftest_nodes['by'] = ftest_nodes.geometry.centroid.y
fosmid2geo = {osmid:geo for osmid, geo in zip(ftest_nodes['osmidstr'], ftest_nodes['geometry'])}

ftest_edges = osm_edges[osm_edges['magnitude'] > 0].to_crs('epsg:3310')
ftest_edges['bx'] = ftest_edges.geometry.centroid.x
ftest_edges['by'] = ftest_edges.geometry.centroid.y

house_gdf_3310 = nhouse_gdf.to_crs('epsg:3310')
house_gdf_3310['bx'] = house_gdf_3310.geometry.centroid.x
house_gdf_3310['by'] = house_gdf_3310.geometry.centroid.y

In [28]:
vectors = []
geos = []
for _, item in tqdm.tqdm(house_gdf_3310.iterrows(), total=len(house_gdf)):
    iftest_edges = ftest_edges[(ftest_edges['bx'] > item['bx'] - 500) & 
                                (ftest_edges['bx'] < item['bx'] + 500) & 
                                (ftest_edges['by'] > item['by'] - 500) & 
                                (ftest_edges['by'] < item['by'] + 500)]
    if len(iftest_edges) == 0:
        iftest_edges = ftest_edges
    
    
    jtem = iftest_edges.iloc[iftest_edges.distance(item.geometry).argmin()]
    u, v = str(jtem['u']), str(jtem['v'])
    vec_u, vec_v = model_node.wv[u], model_node.wv[v]
    dist_u, dist_v = item.geometry.distance(fosmid2geo[u]), item.geometry.distance(fosmid2geo[v])
    ratios = np.array([1/dist_u, 1/dist_v])
    ratios /= np.sum(ratios)
    
    
    vec = vec_u*ratios[0] + vec_v*ratios[1]
    vectors.append(vec)

100%|████████████████████████████████████████████████████████████████████████████| 15368/15368 [00:31<00:00, 493.87it/s]


In [29]:
np.save(f'{dname}/road2vec_n2v_32_nodes.npy', vectors)

In [30]:
house_gdf_3310 = house_gdf.to_crs('epsg:3310')
thouse_gdf_3310 = train_gdf.to_crs('epsg:3310')

In [31]:
n2v_vectors = np.array(vectors)
n2v_vectors_train = n2v_vectors[:len(thouse_gdf_3310)]

In [32]:
train_gdf_3310 = train_gdf.to_crs('epsg:3310')

In [33]:
dist_mat = []
for i, (_, item) in tqdm.tqdm(enumerate(house_gdf_3310.iterrows()), total=len(house_gdf_3310)):
    mdist1 = np.linalg.norm(n2v_vectors[i] - n2v_vectors_train, axis=-1)
    mdist1 /= mdist1.std()
    #dist_mat.append(mdist1)
    mdist2 = train_gdf_3310.distance(item.geometry).values
    mdist2 /= mdist2.std()
    dist_mat.append(mdist1 + mdist2*10)
dist_mat = np.stack(dist_mat, 0)

100%|████████████████████████████████████████████████████████████████████████████| 15368/15368 [00:52<00:00, 291.15it/s]


In [34]:
j=100
ahgdf = train_gdf.copy()
ahgdf['diststr'] = dist_mat[j]

In [35]:
Map([
#     Layer(fhgdf, color_continuous_style('diststr', palette='sunset')),
#     Layer(gdf.iloc[data_ori['idx_geo'][j]], color_continuous_style('price', palette='sunset'))
    Layer(ahgdf.iloc[dist_mat[j].argsort()[1:61]], color_continuous_style('diststr', palette='sunset')),
    Layer(house_gdf.iloc[[j]], basic_style(color='green')),

])

/home/smhan/.conda/envs/TF2/lib/python3.10/site-packages/cartoframes/utils/utils.py:552: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  parameter_args = inspect.getargspec(decorated_function).args


In [36]:
new_idx = []
new_dist_eucli = []
new_dist_geo = []

for i, (_, item) in tqdm.tqdm(enumerate(house_gdf_3310.iterrows()), total=len(house_gdf_3310)):
    target = item.values[:-1]
    candidates = thouse_gdf_3310.iloc[dist_mat[i].argsort()[1:61]].values[:, :-1]
    new_idx.append(dist_mat[i].argsort()[1:61])
    new_dist_geo.append(dist_mat[i][dist_mat[i].argsort()[1:61]])
    new_dist_eucli.append(np.linalg.norm(target.astype(float) - candidates.astype(float), axis=-1))
    
new_idx = np.stack(new_idx, 0)
new_dist_eucli = np.stack(new_dist_eucli, 0)
new_dist_geo = np.stack(new_dist_geo, 0)

100%|████████████████████████████████████████████████████████████████████████████| 15368/15368 [00:47<00:00, 326.68it/s]


In [37]:
newdata = dict() 
newdata['X_train'] = data['X_train']
newdata['y_train'] = data['y_train']
newdata['X_test'] = data['X_test']
newdata['y_test'] = data['y_test']
newdata['idx_eucli'] = new_idx
newdata['idx_geo'] = new_idx
newdata['dist_eucli'] = new_dist_eucli
newdata['dist_geo'] = new_dist_geo

In [38]:
dname

'poa'

In [39]:
np.savez(f'{dname}/data_n2v_nodes.npz', **newdata)